# ElliCE: Efficient and Provably Robust Algorithmic Recourse

This notebook demonstrates the features of the **ElliCE** library, as described in the [README](README.md).

**ElliCE** generates provably robust counterfactual explanations ensuring validity across the Rashomon set of nearly-optimal models.

## Table of Contents
1. [Installation & Setup](#setup)
2. [Quick Start](#quick-start)
3. [Advanced Actionability Constraints](#constraints)
   - Immutable Features
   - Range Constraints
   - One-Way Changes
   - Categorical Features
4. [Generators](#generators)
   - Continuous (with Sparsity)
   - Data-Supported
5. [Custom Backend (PyTorch)](#custom-backend)

## 1. Installation & Setup <a id="setup"></a>

Ensure `ellice` is installed. If you are running this from the repo, you can install it in editable mode.

In [1]:
# !pip install ellice
# Or if running from repo source:
# !pip install -e .

In [ ]:
import sys
import os

# Ensure display is defined if running in non-IPython environment (fallback)
try:
    from IPython.display import display, HTML
except ImportError:
    def display(*args):
        for arg in args:
            print(arg)
    def HTML(text):
        return text

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import ellice
from ellice.configs import GenerationConfig, AlgorithmConfig

# Global Configuration
# In practive this robustness_epsilon could be set to 10% of train loss as default, or determited used set of proxi models (hyperparameter tuning is procedure described in the paper)
# Here we use additive 0.01 to the train loss, so we have loss <= train loss + 0.01, 10% of train loss in practise would mean loss <= train loss * 1.1
robustness_epsilon = 0.01
regularization_coefficient = 0.005

# Reproducibility
def seed_everything(seed: int):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

seed_everything(42)

# Helper function to display query vs CF with highlighted changes
def display_query_vs_cf(query: pd.Series, cf: pd.Series, feature_names: list, threshold: float = 1e-4, 
                        explainer=None, target_class=None, robustness_epsilon=robustness_epsilon, regularization_coefficient=regularization_coefficient):
    """Display query and CF side by side, highlighting changed features.
    
    Args:
        query: Original query instance
        cf: Counterfactual instance
        feature_names: List of feature names
        threshold: Threshold for considering a feature changed
        explainer: Optional ElliCE Explainer instance for computing predictions
        target_class: Optional target class for robust probability calculation
        robustness_epsilon: Epsilon value for robust probability calculation
        regularization_coefficient: Regularization coefficient for robust probability calculation
    """
    try:
        from IPython.display import HTML
        diff = cf[feature_names] - query[feature_names]
        changed_features = diff[diff.abs() > threshold]
        
        # Create HTML table
        html = "<table border='1' style='border-collapse: collapse;'>"
        html += "<tr><th>Feature</th><th>Original</th><th>Counterfactual</th><th>Change</th></tr>"
        
        for feat in feature_names:
            orig_val = query[feat]
            cf_val = cf[feat]
            change = diff[feat]
            is_changed = abs(change) > threshold
            
            color = "#cce5ff" if is_changed else "#ffffff"
            html += f"<tr style='background-color: {color}'>"
            html += f"<td>{feat}</td>"
            html += f"<td>{orig_val:.4f}</td>"
            html += f"<td>{cf_val:.4f}</td>"
            html += f"<td>{change:.4f}</td>"
            html += "</tr>"
        
        html += "</table>"
        display(HTML(html))
        
        # Print prediction information if explainer is provided
        if explainer is not None:
            cf_features = pd.DataFrame([cf[feature_names]], columns=feature_names)
            
            # Get model probability (original model)
            model_probs = explainer.model.predict_proba(cf_features.values)
            predicted_class = 1 if model_probs[0, 1] > 0.5 else 0
            
            # Get probability for target class
            if target_class is not None:
                predicted_prob = model_probs[0, target_class]
            else:
                predicted_prob = model_probs[0, predicted_class]
            
            # Get robust probability (worst case model)
            robust_prob = predicted_prob  # Default fallback
            if target_class is not None:
                try:
                    from ellice.generators.continuous import ContinuousGenerator
                    temp_gen = ContinuousGenerator(
                        model=explainer.model,
                        data=explainer.data,
                        eps=robustness_epsilon,
                        reg_coef=regularization_coefficient,
                        device=str(explainer.device)
                    )
                    robust_probs = temp_gen.get_worst_case_prob(cf_features, target_class=target_class)
                    robust_prob = robust_probs[0]
                except Exception as e:
                    # Fallback: just show model prob
                    robust_prob = predicted_prob
            
            print(f"\nPrediction Information:")
            print(f"  Predicted Class: {predicted_class}")
            print(f"  Predicted Probability of Target Class: {predicted_prob:.4f}")
            if target_class is not None:
                print(f"  Robust Probability (Worst Case) of Target Class: {robust_prob:.4f}")
        
    except:
        # Fallback to simple print if HTML fails
        print("\n=== Original Query ===")
        print(query[feature_names])
        print("\n=== Counterfactual ===")
        print(cf[feature_names])
        diff = cf[feature_names] - query[feature_names]
        changed = diff[diff.abs() > threshold]
        print(f"\n=== Changed Features ({len(changed)}) ===")
        print(changed)
        
        # Print prediction info in fallback mode too
        if explainer is not None:
            cf_features = pd.DataFrame([cf[feature_names]], columns=feature_names)
            model_probs = explainer.model.predict_proba(cf_features.values)
            predicted_class = 1 if model_probs[0, 1] > 0.5 else 0
            # Get probability for target class
            if target_class is not None:
                predicted_prob = model_probs[0, target_class]
            else:
                predicted_prob = model_probs[0, predicted_class]
            print(f"\nPrediction Information:")
            print(f"  Predicted Class: {predicted_class}")
            print(f"  Predicted Probability of Target Class: {predicted_prob:.4f}")

## 2. Quick Start <a id="quick-start"></a>

We'll use the Breast Cancer dataset and a simple Logistic Regression model.

In [3]:
# 1. Load Data
data_raw = load_breast_cancer()
X = pd.DataFrame(data_raw.data, columns=data_raw.feature_names)
y = pd.Series(data_raw.target, name="target")

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#When processing the data, preferably we need to normalize it, butfor simplicity, we will not do that
#The implication is that we will use larger regularization_coefficient for the robust probability calculation stability

# 2. Train Model
clf = LogisticRegression(max_iter=5000, solver='liblinear').fit(X_train, y_train)
print(f"Model Accuracy: {clf.score(X_test, y_test):.4f}")

# 3. Initialize ElliCE
# We need a dataframe that includes the target for ElliCE's Data object
full_df = X_train.copy()
full_df['target'] = y_train

data = ellice.Data(dataframe=full_df, target_column='target')

exp = ellice.Explainer(
    model=clf,
    data=data,
    backend='sklearn',
    device='auto'  # Automatically selects CUDA/MPS if available. Use 'cpu' if you encounter CUDA errors.
)

Model Accuracy: 0.9561


### PyTorch Backend Example
We can also use a PyTorch neural network model with ElliCE.


In [4]:
# 1. Define PyTorch Model
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, 32)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(32, 1)  # Binary output (logits)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x

# 3. Train PyTorch Model
torch_model = SimpleNN(input_dim=X_train.shape[1])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch_model = torch_model.to(device)

# Convert data to tensors
X_train_t = torch.FloatTensor(X_train.values).to(device)
y_train_t = torch.FloatTensor(y_train.values).unsqueeze(1).to(device)
X_test_t = torch.FloatTensor(X_test.values).to(device)
y_test_t = torch.FloatTensor(y_test.values).unsqueeze(1).to(device)

# Training loop
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(torch_model.parameters(), lr=0.001)
epochs = 100

torch_model.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = torch_model(X_train_t)
    loss = criterion(outputs, y_train_t)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 20 == 0:
        with torch.no_grad():
            torch_model.eval()
            test_outputs = torch_model(X_test_t)
            test_preds = (torch.sigmoid(test_outputs) > 0.5).float()
            accuracy = (test_preds == y_test_t).float().mean().item()
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Test Accuracy: {accuracy:.4f}")
            torch_model.train()

torch_model.eval()
print(f"Final PyTorch Model Accuracy: {accuracy:.4f}")

# 4. Use with ElliCE
exp_torch_quick = ellice.Explainer(
    model=torch_model,
    data=data,
    backend='pytorch',
    #backend_model_class=QuickStartModelWrapper,
    device='auto'
)

# Generate CF with PyTorch model
query_torch = X_test.iloc[0]
original_pred_torch = (torch.sigmoid(torch_model(torch.FloatTensor(query_torch.values).unsqueeze(0).to(device))) > 0.5).item()
target_class_torch = 1 - int(original_pred_torch)

print(f"\nPyTorch Model - Original Prediction: {int(original_pred_torch)}")
print(f"PyTorch Model - Target Prediction: {target_class_torch}")

cf_torch_quick = exp_torch_quick.generate_counterfactuals(
    query_instances=query_torch,
    method='continuous',
    target_class=target_class_torch,
    robustness_epsilon=robustness_epsilon,
    regularization_coefficient=regularization_coefficient
)

if not cf_torch_quick.empty:
    print("\nPyTorch Counterfactual Found!")
    display_query_vs_cf(query_torch, cf_torch_quick.iloc[0], data.feature_names,
                       explainer=exp_torch_quick, target_class=target_class_torch,
                       robustness_epsilon=robustness_epsilon, regularization_coefficient=regularization_coefficient)


Epoch 20/100, Loss: 0.5206, Test Accuracy: 0.8158
Epoch 40/100, Loss: 0.2844, Test Accuracy: 0.9298
Epoch 60/100, Loss: 0.2578, Test Accuracy: 0.9474
Epoch 80/100, Loss: 0.2452, Test Accuracy: 0.9561
Epoch 100/100, Loss: 0.2363, Test Accuracy: 0.9649
Final PyTorch Model Accuracy: 0.9649

PyTorch Model - Original Prediction: 0
PyTorch Model - Target Prediction: 1
Progress Bar Enabled


Generating CF:   9%|▉         | 89/1000 [00:00<00:00, 1021.71it/s, Prob=0.730, RobLogit=-0.000, BestRobLogit=0.000]


PyTorch Counterfactual Found!


Feature,Original,Counterfactual,Change
mean radius,12.4700,13.0353,0.5653
mean texture,18.6000,22.6056,4.0056
mean perimeter,81.0900,87.2303,6.1403
mean area,481.9000,490.4293,8.5293
mean smoothness,0.0997,0.1634,0.0637
mean compactness,0.1058,0.1632,0.0574
mean concavity,0.0800,0.0000,-0.0800
mean concave points,0.0382,0.0000,-0.0382
mean symmetry,0.1925,0.3040,0.1115
mean fractal dimension,0.0637,0.0974,0.0337



Prediction Information:
  Predicted Class: 1
  Predicted Probability of Target Class: 0.7300
  Robust Probability (Worst Case) of Target Class: 0.5001


### Generate a Robust Counterfactual
We pick a query instance and generate a counterfactual that flips the prediction.

In [5]:
query = X_test.iloc[0]
original_pred = clf.predict([query])[0]
target_class = 1 - original_pred

print(f"Original Prediction: {original_pred} ({'Malignant' if original_pred==0 else 'Benign'})")
print(f"Target Prediction:   {target_class} ({'Malignant' if target_class==0 else 'Benign'})")

# Generate CF
try:
    cf = exp.generate_counterfactuals(
        query_instances=query,
        method='continuous',
        target_class=target_class,
        robustness_epsilon=robustness_epsilon,
        regularization_coefficient=regularization_coefficient,
        features_to_vary='all',
        return_probs=True
    )

    # Display results
    if not cf.empty:
        print("\nCounterfactual Found!")
        display_query_vs_cf(query, cf.iloc[0], data.feature_names,
                          explainer=exp, target_class=target_class,
                          robustness_epsilon=robustness_epsilon, regularization_coefficient=regularization_coefficient)
    else:
        print("No counterfactual found.")
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

/Users/bt4811/anaconda3/envs/ellice/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Original Prediction: 1 (Benign)
Target Prediction:   0 (Malignant)
Progress Bar Enabled


Generating CF:   0%|          | 3/1000 [00:00<00:01, 649.21it/s, Prob=0.850, RobLogit=0.093, BestRobLogit=0.093]  


Counterfactual Found!



/Users/bt4811/anaconda3/envs/ellice/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Feature,Original,Counterfactual,Change
mean radius,12.4700,12.1697,-0.3003
mean texture,18.6000,18.3005,-0.2995
mean perimeter,81.0900,81.3127,0.2227
mean area,481.9000,482.2005,0.3005
mean smoothness,0.0997,0.1634,0.0637
mean compactness,0.1058,0.3114,0.2056
mean concavity,0.0800,0.3778,0.2978
mean concave points,0.0382,0.2012,0.1630
mean symmetry,0.1925,0.3040,0.1115
mean fractal dimension,0.0637,0.0974,0.0337



Prediction Information:
  Predicted Class: 0
  Predicted Probability of Target Class: 0.8504
  Robust Probability (Worst Case) of Target Class: 0.5233


/Users/bt4811/anaconda3/envs/ellice/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


## 3. Advanced Actionability Constraints <a id="constraints"></a>

ElliCE supports various constraints to make counterfactuals realistic.

### Immutable Features
Prevent features like 'mean radius' from changing.

In [6]:
feature_to_freeze = ['mean radius', 'texture error', 'concavity error', 'symmetry error', 'fractal dimension error']
features_to_vary = [col for col in X.columns if col not in feature_to_freeze]

cf_immutable = exp.generate_counterfactuals(
    query_instances=query,
    method='continuous',
    target_class=target_class,
    features_to_vary=features_to_vary,
    robustness_epsilon=robustness_epsilon,
    regularization_coefficient=regularization_coefficient
)

if not cf_immutable.empty:
    print("\nImmutable Features Counterfactual:")
    display_query_vs_cf(query, cf_immutable.iloc[0], data.feature_names,
                      explainer=exp, target_class=target_class,
                      robustness_epsilon=robustness_epsilon, regularization_coefficient=regularization_coefficient)
    
    # Verify immutable feature
    original_val = query[feature_to_freeze]
    cf_val = cf_immutable.iloc[0][feature_to_freeze]
    print(f"\nVerification - {feature_to_freeze}:")
    print(f"  Original: {original_val}")
    print(f"  CF:       {cf_val}")
    print(f"  Changed? {abs(original_val - cf_val) > 1e-5}")

Progress Bar Enabled


Generating CF:   1%|          | 6/1000 [00:00<00:01, 856.07it/s, Prob=0.904, RobLogit=0.337, BestRobLogit=0.337]   


Immutable Features Counterfactual:


Feature,Original,Counterfactual,Change
mean radius,12.4700,12.4700,0.0000
mean texture,18.6000,18.0045,-0.5955
mean perimeter,81.0900,81.5254,0.4354
mean area,481.9000,482.5030,0.6030
mean smoothness,0.0997,0.1634,0.0637
mean compactness,0.1058,0.3114,0.2056
mean concavity,0.0800,0.4268,0.3468
mean concave points,0.0382,0.2012,0.1630
mean symmetry,0.1925,0.3040,0.1115
mean fractal dimension,0.0637,0.0974,0.0337



Prediction Information:
  Predicted Class: 0
  Predicted Probability of Target Class: 0.9044
  Robust Probability (Worst Case) of Target Class: 0.5833

Verification - ['mean radius', 'texture error', 'concavity error', 'symmetry error', 'fractal dimension error']:
  Original: mean radius                12.470000
texture error               1.044000
concavity error             0.027010
symmetry error              0.017820
fractal dimension error     0.003586
Name: 204, dtype: float64
  CF:       mean radius                12.470000
texture error               1.044000
concavity error             0.027010
symmetry error              0.017820
fractal dimension error     0.003586
Name: 0, dtype: float64
  Changed? mean radius                False
texture error              False
concavity error            False
symmetry error             False
fractal dimension error    False
dtype: bool


/Users/bt4811/anaconda3/envs/ellice/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


### Range Constraints & One-Way Changes
Restrict `mean texture` to a specific range and force `mean area` to only increase.

In [7]:
# Setup constraints
ranges = {'mean texture': [10.0, 25.0]}
one_way = {'mean area': 'increase'}

cf_constrained = exp.generate_counterfactuals(
    query_instances=query,
    method='continuous',
    target_class=target_class,
    permitted_range=ranges,
    one_way_change=one_way,
    robustness_epsilon=robustness_epsilon,
    regularization_coefficient=regularization_coefficient
)

if not cf_constrained.empty:
    print("\nConstrained Counterfactual (Range & One-Way):")
    display_query_vs_cf(query, cf_constrained.iloc[0], data.feature_names,
                       explainer=exp, target_class=target_class,
                       robustness_epsilon=robustness_epsilon, regularization_coefficient=regularization_coefficient)
    
    # Verify constraints
    res = cf_constrained.iloc[0]
    print(f"\nConstraint Verification:")
    print(f"  Mean Texture: {res['mean texture']:.4f} (Allowed: {ranges['mean texture']})")
    print(f"  Mean Area: {res['mean area']:.4f} (Original: {query['mean area']:.4f})")

Progress Bar Enabled


Generating CF:   0%|          | 3/1000 [00:00<00:01, 782.62it/s, Prob=0.850, RobLogit=0.093, BestRobLogit=0.093]   


Constrained Counterfactual (Range & One-Way):


Feature,Original,Counterfactual,Change
mean radius,12.4700,12.1697,-0.3003
mean texture,18.6000,18.3005,-0.2995
mean perimeter,81.0900,81.3127,0.2227
mean area,481.9000,482.2005,0.3005
mean smoothness,0.0997,0.1634,0.0637
mean compactness,0.1058,0.3114,0.2056
mean concavity,0.0800,0.3778,0.2978
mean concave points,0.0382,0.2012,0.1630
mean symmetry,0.1925,0.3040,0.1115
mean fractal dimension,0.0637,0.0974,0.0337



Prediction Information:
  Predicted Class: 0
  Predicted Probability of Target Class: 0.8504
  Robust Probability (Worst Case) of Target Class: 0.5233

Constraint Verification:
  Mean Texture: 18.3005 (Allowed: [10.0, 25.0])
  Mean Area: 482.2005 (Original: 481.9000)


/Users/bt4811/anaconda3/envs/ellice/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


### Categorical Features (One-Hot Encoding)
To demonstrate this, let's modify our dataset to include a categorical feature by binning 'mean smoothness' into 'Low', 'Medium', 'High'.

In [8]:
# Create modified dataset with categorical feature
X_cat = X.copy()
X_cat['smoothness_cat'] = pd.cut(X_cat['mean smoothness'], bins=3, labels=['Low', 'Medium', 'High'])
X_cat = X_cat.drop(columns=['mean smoothness'])
X_cat_encoded = pd.get_dummies(X_cat, columns=['smoothness_cat'], prefix='smoothness', dtype=float)

# Identify one-hot columns
one_hot_cols = [c for c in X_cat_encoded.columns if c.startswith('smoothness_')]
one_hot_groups = [one_hot_cols]
print("One-hot groups:", one_hot_groups)

# Retrain model on new data
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_cat_encoded, y, test_size=0.2, random_state=42)
clf_c = LogisticRegression(max_iter=5000, solver='lbfgs').fit(X_train_c, y_train_c)
print(f"Categorical Model Accuracy: {clf_c.score(X_test_c, y_test_c):.4f}")

# New Explainer
full_df_c = X_train_c.copy()
full_df_c['target'] = y_train_c
data_c = ellice.Data(dataframe=full_df_c, target_column='target')
exp_c = ellice.Explainer(clf_c, data_c, backend='sklearn')

# Generate CF with categorical handling
query_c = X_test_c.iloc[0]
cf_cat = exp_c.generate_counterfactuals(
    query_instances=query_c,
    method='continuous',
    target_class=1 - clf_c.predict([query_c])[0],
    robustness_epsilon=robustness_epsilon,
    regularization_coefficient=regularization_coefficient,
    one_hot_groups=one_hot_groups
)

if not cf_cat.empty:
    print("\nCategorical Features Counterfactual:")
    target_class_c = 1 - clf_c.predict([query_c])[0]
    display_query_vs_cf(query_c, cf_cat.iloc[0], data_c.feature_names,
                       explainer=exp_c, target_class=target_class_c,
                       robustness_epsilon=robustness_epsilon, regularization_coefficient=regularization_coefficient)
    
    print("\nCategorical Feature Values (Should sum to 1):")
    print(cf_cat.iloc[0][one_hot_cols])
    print("Sum:", cf_cat.iloc[0][one_hot_cols].sum())

One-hot groups: [['smoothness_Low', 'smoothness_Medium', 'smoothness_High']]


/Users/bt4811/anaconda3/envs/ellice/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Categorical Model Accuracy: 0.9737
Progress Bar Enabled


Generating CF:   1%|          | 7/1000 [00:00<00:01, 769.92it/s, Prob=0.942, RobLogit=0.210, BestRobLogit=0.210]  



Categorical Features Counterfactual:


/Users/bt4811/anaconda3/envs/ellice/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Feature,Original,Counterfactual,Change
mean radius,12.4700,11.7849,-0.6851
mean texture,18.6000,17.9027,-0.6973
mean perimeter,81.0900,81.5300,0.4400
mean area,481.9000,482.3299,0.4299
mean compactness,0.1058,0.3049,0.1991
mean concavity,0.0800,0.4268,0.3468
mean concave points,0.0382,0.2012,0.1630
mean symmetry,0.1925,0.3040,0.1115
mean fractal dimension,0.0637,0.0852,0.0215
radius error,0.3961,0.4844,0.0883


/Users/bt4811/anaconda3/envs/ellice/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(



Prediction Information:
  Predicted Class: 0
  Predicted Probability of Target Class: 0.9417
  Robust Probability (Worst Case) of Target Class: 0.5523

Categorical Feature Values (Should sum to 1):
smoothness_Low       0.0
smoothness_Medium    1.0
smoothness_High      0.0
Name: 0, dtype: float64
Sum: 1.0


## 4. Generators <a id="generators"></a>

### Continuous Generator with Sparsity
Try to minimize the number of features changed.

In [9]:
cf_sparse = exp.generate_counterfactuals(
    query_instances=query,
    method='continuous',
    target_class=target_class,
    sparsity=True,  # Enable sparsity
    robustness_epsilon=robustness_epsilon,
    regularization_coefficient=regularization_coefficient
)

if not cf_sparse.empty:
    print("\nSparse Counterfactual (Minimal Features Changed):")
    display_query_vs_cf(query, cf_sparse.iloc[0], data.feature_names,
                      explainer=exp, target_class=target_class,
                      robustness_epsilon=robustness_epsilon, regularization_coefficient=regularization_coefficient)
    
    diff = cf_sparse.iloc[0][data.feature_names] - query
    changed = diff[diff.abs() > 1e-4]
    print(f"\nNumber of features changed: {len(changed)}")
    print("Changed features:", changed.index.tolist())

Running Sparse Optimization...
Valid CF found with 2 active features (or groups).

Sparse Counterfactual (Minimal Features Changed):


Feature,Original,Counterfactual,Change
mean radius,12.4700,11.3655,-1.1045
mean texture,18.6000,18.6000,0.0000
mean perimeter,81.0900,81.0900,-0.0000
mean area,481.9000,481.9000,-0.0000
mean smoothness,0.0997,0.0997,0.0000
mean compactness,0.1058,0.1058,0.0000
mean concavity,0.0800,0.0800,-0.0000
mean concave points,0.0382,0.0382,0.0000
mean symmetry,0.1925,0.1925,-0.0000
mean fractal dimension,0.0637,0.0637,0.0000



Prediction Information:
  Predicted Class: 0
  Predicted Probability of Target Class: 0.9110
  Robust Probability (Worst Case) of Target Class: 0.5189

Number of features changed: 2
Changed features: ['mean radius', 'worst concavity']


/Users/bt4811/anaconda3/envs/ellice/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


With freezed features

In [13]:
feature_to_freeze = ['mean radius']
features_to_vary = [col for col in X.columns if col not in feature_to_freeze]

cf_sparse = exp.generate_counterfactuals(
    query_instances=query,
    method='continuous',
    target_class=target_class,
    sparsity=True,  # Enable sparsity
    robustness_epsilon=robustness_epsilon,
    regularization_coefficient=regularization_coefficient,
    features_to_vary=features_to_vary
)

if not cf_sparse.empty:
    print("\nSparse Counterfactual (Minimal Features Changed):")
    display_query_vs_cf(query, cf_sparse.iloc[0], data.feature_names,
                      explainer=exp, target_class=target_class,
                      robustness_epsilon=robustness_epsilon, regularization_coefficient=regularization_coefficient)
    
    diff = cf_sparse.iloc[0][data.feature_names] - query
    changed = diff[diff.abs() > 1e-4]
    print(f"\nNumber of features changed: {len(changed)}")
    print("Changed features:", changed.index.tolist())

Running Sparse Optimization...
Valid CF found with 6 active features (or groups).

Sparse Counterfactual (Minimal Features Changed):


Feature,Original,Counterfactual,Change
mean radius,12.4700,12.4700,0.0000
mean texture,18.6000,18.6000,0.0000
mean perimeter,81.0900,81.0900,-0.0000
mean area,481.9000,481.9000,-0.0000
mean smoothness,0.0997,0.0997,0.0000
mean compactness,0.1058,0.1058,0.0000
mean concavity,0.0800,0.4268,0.3468
mean concave points,0.0382,0.0382,0.0000
mean symmetry,0.1925,0.1925,-0.0000
mean fractal dimension,0.0637,0.0637,0.0000



Prediction Information:
  Predicted Class: 0
  Predicted Probability of Target Class: 0.9073
  Robust Probability (Worst Case) of Target Class: 0.5073

Number of features changed: 6
Changed features: ['mean concavity', 'texture error', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry']


/Users/bt4811/anaconda3/envs/ellice/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [10]:
# cf_sparse.shape

### Data-Supported Generator
Finds a counterfactual from the actual training data.

In [11]:
cf_data = exp.generate_counterfactuals(
    query_instances=query,
    method='data_supported',
    search_mode='kdtree',  # Fast search
    target_class=target_class
)

if not cf_data.empty:
    print("\nData-Supported Counterfactual:")
    display_query_vs_cf(query, cf_data.iloc[0], data.feature_names,
                      explainer=exp, target_class=target_class)
    
    # Verify it's a real point (check index or exact match)
    is_real = (X_train == cf_data.iloc[0][data.feature_names]).all(axis=1).any()
    print(f"\nIs this a real data point? {is_real}")


Data-Supported Counterfactual:


Feature,Original,Counterfactual,Change
mean radius,12.4700,13.0000,0.5300
mean texture,18.6000,21.8200,3.2200
mean perimeter,81.0900,87.5000,6.4100
mean area,481.9000,519.8000,37.9000
mean smoothness,0.0997,0.1273,0.0276
mean compactness,0.1058,0.1932,0.0874
mean concavity,0.0800,0.1859,0.1059
mean concave points,0.0382,0.0935,0.0553
mean symmetry,0.1925,0.2350,0.0425
mean fractal dimension,0.0637,0.0739,0.0102



Prediction Information:
  Predicted Class: 0
  Predicted Probability of Target Class: 0.9076
  Robust Probability (Worst Case) of Target Class: 0.6839

Is this a real data point? True


/Users/bt4811/anaconda3/envs/ellice/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


## 5. Custom Backend (PyTorch) <a id="custom-backend"></a>

Defining a custom model wrapper for a PyTorch model.

In [12]:
from ellice.models.wrappers import ModelWrapper
from typing import Tuple

# 1. Define PyTorch Model
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, 16)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(16, 1) # Binary output (logits)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x

# 2. Create Wrapper
class MyModelWrapper(ModelWrapper):
    def __init__(self, model):
        super().__init__(model, backend='custom')
        self.model.eval()
        
    def get_torch_model(self) -> nn.Module:
        return self.model
        
    def split_model(self) -> Tuple[nn.Module, torch.Tensor]:
        # Split into penultimate features (layer1+relu) and last layer (layer2)
        penult = nn.Sequential(self.model.layer1, self.model.relu)
        
        # Get last layer params [weights, bias]
        last = self.model.layer2
        theta = torch.cat([last.weight.detach().view(-1), last.bias.detach()])
        return penult, theta
        
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        device = next(self.model.parameters()).device
        X_t = torch.from_numpy(X).float().to(device)
        with torch.no_grad():
            logits = self.model(X_t)
            probs_1 = torch.sigmoid(logits)
            probs_0 = 1 - probs_1
            return torch.cat([probs_0, probs_1], dim=1).cpu().numpy()

# 3. Setup and Train
torch_model = SimpleNN(input_dim=X_train.shape[1])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch_model = torch_model.to(device)

# Convert data to tensors
X_train_t = torch.FloatTensor(X_train.values).to(device)
y_train_t = torch.FloatTensor(y_train.values).unsqueeze(1).to(device)
X_test_t = torch.FloatTensor(X_test.values).to(device)
y_test_t = torch.FloatTensor(y_test.values).unsqueeze(1).to(device)

# Training loop
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(torch_model.parameters(), lr=0.001)
epochs = 100

print("Training PyTorch model...")
torch_model.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = torch_model(X_train_t)
    loss = criterion(outputs, y_train_t)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 20 == 0:
        with torch.no_grad():
            torch_model.eval()
            test_outputs = torch_model(X_test_t)
            test_preds = (torch.sigmoid(test_outputs) > 0.5).float()
            accuracy = (test_preds == y_test_t).float().mean().item()
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Test Accuracy: {accuracy:.4f}")
            torch_model.train()

torch_model.eval()
print(f"Training complete! Final Accuracy: {accuracy:.4f}")

# 4. Use with ElliCE
exp_torch = ellice.Explainer(
    model=torch_model,
    data=data,
    backend='custom',
    backend_model_class=MyModelWrapper,
    device='auto'
)

print("Wrapper initialized successfully!")

# Generate CF using the custom PyTorch backend
cf_torch = exp_torch.generate_counterfactuals(
    query_instances=query,
    method='continuous',
    target_class=target_class,
    robustness_epsilon=robustness_epsilon,
    regularization_coefficient=regularization_coefficient
)

if not cf_torch.empty:
    print("\nCustom PyTorch Backend Counterfactual:")
    display_query_vs_cf(query, cf_torch.iloc[0], data.feature_names,
                       explainer=exp_torch, target_class=target_class,
                       robustness_epsilon=robustness_epsilon, regularization_coefficient=regularization_coefficient)

Training PyTorch model...
Epoch 20/100, Loss: 1.2975, Test Accuracy: 0.6491
Epoch 40/100, Loss: 0.5844, Test Accuracy: 0.8158
Epoch 60/100, Loss: 0.3606, Test Accuracy: 0.9035
Epoch 80/100, Loss: 0.3439, Test Accuracy: 0.9123
Epoch 100/100, Loss: 0.3224, Test Accuracy: 0.9123
Training complete! Final Accuracy: 0.9123
Wrapper initialized successfully!
Progress Bar Enabled


Generating CF:  10%|▉         | 97/1000 [00:00<00:00, 915.76it/s, Prob=0.703, RobLogit=0.014, BestRobLogit=0.014]   


Custom PyTorch Backend Counterfactual:


Feature,Original,Counterfactual,Change
mean radius,12.4700,12.2708,-0.1992
mean texture,18.6000,10.0003,-8.5997
mean perimeter,81.0900,72.2290,-8.8610
mean area,481.9000,490.9725,9.0725
mean smoothness,0.0997,0.0526,-0.0470
mean compactness,0.1058,0.0194,-0.0864
mean concavity,0.0800,0.4268,0.3468
mean concave points,0.0382,0.2012,0.1630
mean symmetry,0.1925,0.1167,-0.0758
mean fractal dimension,0.0637,0.0974,0.0337



Prediction Information:
  Predicted Class: 0
  Predicted Probability of Target Class: 0.7029
  Robust Probability (Worst Case) of Target Class: 0.5035
